In [10]:
#This class store the internsity and pitch 
class Pause_Info:
  def __init__(self,intensity,stinten,pitch,stpit,max_inten,min_inten,max_pitch,min_pitch,performance,id,jokeName):
    self.intensity = intensity
    self.stinten= stinten
    self.pitch = pitch
    self.stpit= stpit
    self.max_inten=max_inten
    self.min_inten=min_inten
    self.max_pitch=max_pitch
    self.min_pitch=min_pitch
    self.performance=performance
    self.id=id
    self.jokeName=jokeName
    
    
#this function loop through the folder "joke_name_matching" and make a dictionary of 
#joke id and joke name for each performance
def getJokeNameForEachPerformance(folderPath):
    os.chdir(folderPath)
    
    jokeNameDict={}
    
    for file in glob.glob("*.txt"):
        idDict={}
        filepath=folderPath+'\\'+file
        f = open(filepath, "r")
        key=f.readline()
        key=key.replace("\x00","")
        key=key.replace("\n","")
#         print(key)
        content=f.read();
        f.close()
        content=content.replace("\x00","")
        content=content.replace("\n\n","\n")
        if not content:
            continue
        lines=content.split('\n')
        lines.pop(0)
        lines.pop(-1)
        for line in lines:
            jokename=line.split(':')[0]
            id=int(line.split(':')[1].split('.')[0].split('_')[1])
#             print(jokename,id)
            idDict[id]=jokename
        jokeNameDict[key]=idDict
    
    return jokeNameDict
            

#This function extract intensity and pitch info from txt file and put all info in to a dictionary
def readTxT(path):
    f = open(path, "r")
    line = f.readline()
    
    #store the joke title
    Title = line.split('\\')[-2]
    Title=Title.replace("\x00","")
    joke_dict = {}

    content=f.read();
    f.close()
    
    #replace all these werid \x00 in python
    #also replace all the extra newline char
    content=content.replace("\x00","")
    content=content.replace("\n\n","\n")
    lines=content.split('\n')
    
    #remove the first and last extra new line char
    lines.pop(0)
    lines.pop(-1)

    count=0
    intensity=0
    stinten=0
    pitch=0
    stpit=0
    max_inten=0
    min_inten=0
    max_pitch=0
    min_pitch=0
    digits=2
    
    #store 7 info of each audio
    for line in lines:
        if(count%9==0):
            name=line.split('.')[0]
            name=name.split('_')[-1]
            id=int(name)
        elif(count%9==1):
            intensity=round(float(line),digits)
        elif(count%9==2):
            stinten=round(float(line),digits)
        elif(count%9==3):
            min_inten=round(float(line),digits)
        elif(count%9==4):
            max_inten=round(float(line),digits)
        elif(count%9==5):
            pitch=round(float(line),digits)
        elif(count%9==6):
            stpit=round(float(line),digits)   
        elif(count%9==7):
            max_pitch=round(float(line),digits)
        else:
            min_pitch=round(float(line),digits)
            if(Title not in jokeNameDict):
                break
            idDict=jokeNameDict[Title]
            jokeName=idDict[id]
            #print(jokeName)
            joke_dict[jokeName]=Pause_Info(intensity,stinten,pitch,stpit,max_inten,min_inten,max_pitch,min_pitch,Title,id,jokeName)
           # print(Title,id)
        count+=1
    return joke_dict


import glob, os



#in combine_joke_dict,
#key of combine_joke_dict represent the joke id
#value of combine_joke_dict is an array of 'Pause_Info' who are the same joke but in different perofrmance
#len of value represent how many different perofrmance contain this joke 
def findAlltxtAndCombine(folderPath):
    os.chdir(folderPath)
    combine_joke_dict={}
    
    #find all txt file inside folder
    for file in glob.glob("*.txt"):
        filepath=folderPath+'\\'+file
        single_joke_dict=readTxT(filepath)   
        for x in single_joke_dict:
            y=single_joke_dict[x]
            if x in combine_joke_dict:
                combine_joke_dict[x].append(y)
            else:
                combine_joke_dict[x]=[y]
    return combine_joke_dict         

In [17]:
%matplotlib inline

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import scale
from numpy import random, float
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus']=False

#info_arr represent an array of same joke in different performances
#choice=1 is using intensity and pitch as x-y axis
#choice=2 is using std intensity and std pitch as x-y axis
def clusterDataIn2D(Info_arr,choice):
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    performanceName=[]
    Xname,Yname='',''
    if(choice==1):
        for joke in Info_arr:
            X=joke.intensity
            Y=joke.pitch
            data.append((X,Y))
            performanceName.append(joke.performance)
            x.append(X)
            y.append(Y)
            Xname="intensity"
            Yname="pitch"
    elif(choice==2):
        for joke in Info_arr:
            X=joke.stinten
            Y=joke.stpit
            data.append((X,Y))
            performanceName.append(joke.performance)
            x.append(X)
            y.append(Y)
            Xname="standard deviation intensity"
            Yname="standard deviation pitch"
    
    #if the data set is smller than 2,we will ignore it.
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1

    dataTitle="Data values: ("+Xname+","+Yname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    #print(jokeid)
    txtName="../jokeoutput/2D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
#     print(jokeid)
#     print (model.cluster_centers_)
    
    plt.figure(figsize=(10,6))
    plt.scatter(x, y, c=model.labels_.astype(float))
    plt.title("Joke Name: "+str(jokeid))
    plt.xlabel(Xname)
    plt.ylabel(Yname)
    
    pngName="../jokeoutput/2D/"+str(jokeid)+".png"
    
    #remove the existing png file
    if os.path.isfile(pngName):
        os.remove(pngName) 
    
    #save as png file
    plt.savefig(pngName)
    plt.show()
    

#this function will split data into 2 groups by using 4 features
def clusterDataIn4D(Info_arr):
    #replace all the '/' to '-' since it will mess up with path of windows
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    z=[]
    h=[]
    performanceName=[]
    Xname,Yname='',''
    
    #get all intensity,pitch,std intensity,std pitch
    for joke in Info_arr:
        X=joke.intensity
        Y=joke.pitch
        Z=joke.stinten
        H=joke.stpit
        data.append((X,Y,Z,H))
        performanceName.append(joke.performance)
        x.append(X)
        y.append(Y)
        z.append(Z)
        h.append(H)
        Xname="intensity"
        Yname="pitch"
        Zname="std intensity"
        Hname="std pitch"

    
    #if data set is smaller than 2, we cannot split them into 2groups
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1
    
    dataTitle="Data values: ("+Xname+","+Yname+","+Zname+","+Hname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    txtName="../jokeoutput/4D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
#     print(jokeid)
#     print (model.cluster_centers_)


#this function will split data into 2 groups by using 6 features
def clusterDataIn8D(Info_arr):
    #replace all the '/' to '-' since it will mess up with path of windows
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    z=[]
    d=[]
    e=[]
    f=[]
    g=[]
    h=[]
    performanceName=[]
    Xname,Yname='',''
    
    #get all intensity,pitch,std intensity,std pitch
    for joke in Info_arr:
        X=joke.intensity
        Y=joke.pitch
        Z=joke.stinten
        D=joke.stpit
        E=joke.max_inten
        F=joke.min_inten
        G=joke.max_pitch
        H=joke.min_pitch
 
        data.append((X,Y,Z,D,E,F,G,H))
        performanceName.append(joke.performance+"----Joke ID: "+str(joke.id))
        x.append(X)
        y.append(Y)
        z.append(Z)
        d.append(d)
        e.append(E)
        f.append(F)
        g.append(G)
        h.append(H)
        
        
#         print(E,F)
        
        Xname="intensity"
        Yname="pitch"
        Zname="std intensity"
        Dname="std pitch"
        Ename="max inten"
        Fname="min inten"
        Gname="max pitch"
        Hname="min pitch"

    
    #if data set is smaller than 2, we cannot split them into 2groups
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1
    
    dataTitle="Data values: ("+Xname+","+Yname+","+Zname+","+Dname+","+Ename+","+Fname+","+Gname+","+Hname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name and Joke ID: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name and Joke ID: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    txtName="../jokeoutput/8D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
#     print(jokeid)
#     print (model.cluster_centers_)

In [18]:
import numpy
from scipy.stats import pearsonr
def overallCorrelation(combine_joke_dict):
    x=[]
    y=[]
    z=[]
    d=[]
    e=[]
    f=[]
    g=[]
    h=[]
    for i in combine_joke_dict:
        arr=combine_joke_dict[i]
        for joke in arr:
            x.append(joke.intensity)
            y.append(joke.pitch)
            z.append(joke.stinten)
            d.append(joke.stpit)
            e.append(joke.max_inten)
            f.append(joke.min_inten)
            g.append(joke.max_pitch)
            h.append(joke.min_pitch)
    
    arr=[x,y,z,d,e,f,g,h]
    result={}
    var=['intensity','pitch','stdIntensity','stdPitch','maxIntensity','minIntensity','maxPitch','minPitch']
    length=len(var)
    
    pontential="The correlation coefficient are higher than 7:\n"
    
    for i in range(0,length-1):
        for j in range(i+1,length):
            #save txt result
            key=var[i]+'-'+var[j]
            result[key]=pearsonr(arr[i],arr[j])[0]
            if(abs(result[key])>0.7):
                pontential=pontential+key+": "+str(result[key])+"\n"
            
            #draw plot
            path="../jokeoutput/overall-correlation/"+key+".png"            
            draw2Dplot(var[i],var[j],arr[i],arr[j],path)

    print(len(result))
    txtName="../jokeoutput/overall-correlation/correlation.txt"
    
    output=''
    for n in result:
        output=output+n+": "+str(result[n])+"\n"

    output=output+"\n\n"+pontential
    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(output)
    f.close()
    
def jokeWiseCorrelation(combine_joke_dict):
    x=[]
    y=[]
    z=[]
    d=[]
    e=[]
    f=[]
    g=[]
    h=[]
    var=['intensity','pitch','stdIntensity','stdPitch','maxIntensity','minIntensity','maxPitch','minPitch']
    length=len(var)
    pontential="The correlation coefficient are higher than 7:\n"
    
    for i in combine_joke_dict:
        arr=combine_joke_dict[i]
        x.clear()
        y.clear()
        z.clear()
        d.clear()
        e.clear()
        f.clear()
        g.clear()
        h.clear()
        jokename=''
        for joke in arr:
            x.append(joke.intensity)
            y.append(joke.pitch)
            z.append(joke.stinten)
            d.append(joke.stpit)
            e.append(joke.max_inten)
            f.append(joke.min_inten)
            g.append(joke.max_pitch)
            h.append(joke.min_pitch) 
            jokename=joke.jokeName
            
        jokename=jokename.replace('/','-')
        arr=[x,y,z,d,e,f,g,h]
        result={}
        dirPath="../jokeoutput/jokewise-correlation/"+jokename

        if not os.path.exists(dirPath):
            os.mkdir(dirPath)
                
        for i in range(0,length-1):
            for j in range(i+1,length):
                #save txt result
                key=var[i]+'-'+var[j]
#                 print(len(arr[i]),len(arr[j]))
                if(len(arr[i])<2):
                    result[key]="Data set is smaller than 2"
                    
                    
                else:    
                    result[key]=pearsonr(arr[i],arr[j])[0]
                    if(abs(result[key])>0.7):
                        pontential=pontential+key+": "+str(result[key])+"\n"
                    #draw plot
                    path=dirPath+"/"+key+".png"           
                    print(path)
                    draw2Dplot(var[i],var[j],arr[i],arr[j],path)

        txtName=dirPath+"/correlation.txt"

        output=''
        for n in result:
            output=output+n+": "+str(result[n])+"\n"

        output=output+"\n\n"+pontential
        if os.path.isfile(txtName):
            os.remove(txtName) 
        file = open(txtName, "a")
        file.write(output)
        file.close()
        pontential="The correlation coefficient are higher than 7:\n"
    
def draw2Dplot(Xname,Yname,x,y,path):
    fig=plt.figure(figsize=(10,6))
    plt.scatter(x, y)
    plt.title(Xname+"-"+Yname)
    plt.xlabel(Xname)
    plt.ylabel(Yname)
    pngName=path
    #remove the existing png file
    if os.path.isfile(pngName):
        os.remove(pngName) 
    #save as png file
    plt.savefig(pngName)
#     plt.show()
    plt.close(fig) 
    
    
def main():
    
    global jokeNameDict
    jokeNameDict=getJokeNameForEachPerformance(r"E:\CapstoneWork\k-means-test_by_Tony\joke_name_matching")
    #i am using directory pitch_inten_avg_std_minmax since it is the test contain most information
    combine_joke_dict=findAlltxtAndCombine(r"E:\CapstoneWork\k-means-test_by_Tony\jokeinput")
    for x in combine_joke_dict:
        for joke in combine_joke_dict[x]:
            print(joke.performance,joke.id)
    #print(len(combine_joke_dict))
#     for x in combine_joke_dict:
#         clusterDataIn2D(combine_joke_dict[x],1)
#     for x in combine_joke_dict:
#         clusterDataIn4D(combine_joke_dict[x])
    for x in combine_joke_dict:
        clusterDataIn8D(combine_joke_dict[x])
#     overallCorrelation(combine_joke_dict)
#     jokeWiseCorrelation(combine_joke_dict)
        
main()
print("done")


2019-04-13 Cienna Nerdy Show at The Drake 0
2019-09-05 Stand-up Science 0
2019-09-06 RoboCom 0
2019-09-19 Bombs Away Cafe 0
2019-09-21 Laugh Track Town USA 0
2019-10-11 Singu-hilarity 0
2019-11-29 Comedy the Musical 0
2019-11-29 Crapshoot 0
2019-12-06 Silent Background Recording 0
2019-04-18 Bombs Away Cafe 0
2019-04-19 Singu-hilarity 0
2019-04-22 Class Performance 0
2019-05-16 Bombs Away Cafe 0
2019-06-19 Trek Theater 0
2019-06-20 Bombs Away Cafe 0
2019-08-15 Bombs Away Cafe 0
2019-08-23 Spectrum 0
2019-04-13 Cienna Nerdy Show at The Drake 1
2019-09-05 Stand-up Science 1
2019-09-06 RoboCom 1
2019-09-21 Laugh Track Town USA 1
2019-10-11 Singu-hilarity 1
2019-11-29 Comedy the Musical 1
2019-11-29 Crapshoot 1
2019-12-06 Silent Background Recording 1
2019-04-19 Singu-hilarity 1
2019-04-22 Class Performance 1
2019-05-16 Bombs Away Cafe 1
2019-06-19 Trek Theater 1
2019-08-15 Bombs Away Cafe 1
2019-08-23 Spectrum 1
2019-04-13 Cienna Nerdy Show at The Drake 10
2019-09-06 RoboCom 12
2019-08-15

done
